In [2]:
import os
import zipfile
import pandas as pd
import requests
from timeit import default_timer as timer

## Load the data from TSE

In [3]:
os.makedirs('datasets/new_data/', exist_ok=True)
DIRPATH = 'datasets/new_data/'

In [4]:
year = 2020
BASEURL = f'https://cdn.tse.jus.br/estatistica/sead/odsele/votacao_candidato_munzona/votacao_candidato_munzona_{year}.zip'

In [5]:
def download_zip_file(url, filename):
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()  # Raise an exception for HTTP errors

        with open(filename, 'wb') as file:
            t_start = timer()
            for it, chunk in enumerate(response.iter_content(chunk_size=1024)):
                print(f"Downloaded {it} KB ({it/1024:.2f} MB)", end='\r', flush=False)
                file.write(chunk)
            t_end = timer()
            print()
            print(f'Took {t_end-t_start:.2f} seconds')

        print(f"Downloaded {filename} successfully!")

    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except requests.exceptions.ConnectionError as conn_err:
        print(f"Error connecting to the URL: {conn_err}")
    except requests.exceptions.Timeout as time_err:
        print(f"Timeout error occurred: {time_err}")
    except requests.exceptions.RequestException as err:
        print(f"Something went wrong: {err}")


In [6]:
def filter_by_state(zipfile_list : list[zipfile.ZipInfo], state='RJ'):
    return list(filter(lambda x: state in x.filename, zipfile_list))[0].filename

In [7]:
for year in range(2004,2028,4):
    BASEURL = f'https://cdn.tse.jus.br/estatistica/sead/odsele/votacao_candidato_munzona/votacao_candidato_munzona_{year}.zip'
    filename = BASEURL.split('/')[-1]
    filepath = os.path.join(DIRPATH,filename)
    download_zip_file(BASEURL, filepath)
    with zipfile.ZipFile(filepath,'r') as zip_ref:
        # os.stat(filepath+'/')
        zip_ref.extract(filter_by_state(zip_ref.filelist), path = DIRPATH)
        print('---')
        # os.mkdir(path+str(ano)+'/') 
    
    # for state in estados:
    #     zip_ref.extract(std_filename+str(ano)+'_'+state+'.txt',path=path+str(ano)+'/')
    #     print("Extraindo {} no diretorio {}".format(std_filename+str(ano)+'_'+state+'.txt',path+str(ano)+'/'))

Downloaded 40276 KB (39.33 MB)
Took 14.39 seconds
Downloaded datasets/new_data/votacao_candidato_munzona_2004.zip successfully!
---
Downloaded 41670 KB (40.69 MB)
Took 10.39 seconds
Downloaded datasets/new_data/votacao_candidato_munzona_2008.zip successfully!
---
Downloaded 56438 KB (55.12 MB)
Took 14.66 seconds
Downloaded datasets/new_data/votacao_candidato_munzona_2012.zip successfully!
---
Downloaded 62523 KB (61.06 MB)
Took 15.38 seconds
Downloaded datasets/new_data/votacao_candidato_munzona_2016.zip successfully!
---
Downloaded 81120 KB (79.22 MB)
Took 21.43 seconds
Downloaded datasets/new_data/votacao_candidato_munzona_2020.zip successfully!
---
Downloaded 197 KB (0.19 MB)
Took 0.08 seconds
Downloaded datasets/new_data/votacao_candidato_munzona_2024.zip successfully!
---


## Process the data

In [34]:
df_2020_rj = pd.read_csv('datasets/new_data/votacao_candidato_munzona_2020_RJ.csv', 
                         delimiter=';', 
                         encoding='iso-8859-1')

In [35]:
df_2020_rj.columns

Index(['DT_GERACAO', 'HH_GERACAO', 'ANO_ELEICAO', 'CD_TIPO_ELEICAO',
       'NM_TIPO_ELEICAO', 'NR_TURNO', 'CD_ELEICAO', 'DS_ELEICAO', 'DT_ELEICAO',
       'TP_ABRANGENCIA', 'SG_UF', 'SG_UE', 'NM_UE', 'CD_MUNICIPIO',
       'NM_MUNICIPIO', 'NR_ZONA', 'CD_CARGO', 'DS_CARGO', 'SQ_CANDIDATO',
       'NR_CANDIDATO', 'NM_CANDIDATO', 'NM_URNA_CANDIDATO',
       'NM_SOCIAL_CANDIDATO', 'CD_SITUACAO_CANDIDATURA',
       'DS_SITUACAO_CANDIDATURA', 'CD_DETALHE_SITUACAO_CAND',
       'DS_DETALHE_SITUACAO_CAND', 'TP_AGREMIACAO', 'NR_PARTIDO', 'SG_PARTIDO',
       'NM_PARTIDO', 'NR_FEDERACAO', 'NM_FEDERACAO', 'SG_FEDERACAO',
       'DS_COMPOSICAO_FEDERACAO', 'SQ_COLIGACAO', 'NM_COLIGACAO',
       'DS_COMPOSICAO_COLIGACAO', 'ST_VOTO_EM_TRANSITO', 'QT_VOTOS_NOMINAIS',
       'NM_TIPO_DESTINACAO_VOTOS', 'QT_VOTOS_NOMINAIS_VALIDOS',
       'CD_SIT_TOT_TURNO', 'DS_SIT_TOT_TURNO'],
      dtype='object')

In [36]:
df_2020_rj.shape

(131270, 44)

In [38]:
df_2020_rj['ANO_ELEICAO'].value_counts()

ANO_ELEICAO
2020    131270
Name: count, dtype: int64

In [39]:
df_2020_rj['DS_CARGO'].value_counts()

DS_CARGO
Vereador    129596
Prefeito      1674
Name: count, dtype: int64

In [43]:
df_2020_rj['NM_MUNICIPIO'].value_counts()

NM_MUNICIPIO
RIO DE JANEIRO           83643
SÃO GONÇALO               6489
DUQUE DE CAXIAS           4788
SÃO JOÃO DE MERITI        2828
NITERÓI                   2760
                         ...  
VARRE-SAI                   59
SÃO JOSÉ DE UBÁ             57
MACUCO                      57
SÃO SEBASTIÃO DO ALTO       36
LAJE DO MURIAÉ              30
Name: count, Length: 92, dtype: int64